In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
df = pd.read_csv('estat_prc_hpi_q.csv')
df.columns = df.columns.str.strip()
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df.head()

,freq,purchase,unit,geo,2005-Q1,2005-Q2,2005-Q3,2005-Q4,2006-Q1,2006-Q2,...,2022-Q2,2022-Q3,2022-Q4,2023-Q1,2023-Q2,2023-Q3,2023-Q4,2024-Q1,2024-Q2,2024-Q3
0,Q,DW_EXST,I10_Q,AT,:,:,:,:,:,:,...,223.26,227.52,217.81,215.58,214.13,212.07,207.28,204.23,211.26,211.98 p
1,Q,DW_EXST,I10_Q,BE,72.31,74.46,78.01,79.97,81.55,83.90,...,143.09,146.87,145.40,146.72,146.18,147.84,149.38,149.82,149.77,152.61 p
2,Q,DW_EXST,I10_Q,BG,72.66,75.77,77.96,79.84,83.62,84.99,...,160.96,167.01,169.23,174.59,179.37,182.01,185.80,199.15,205.93,212.00 p
3,Q,DW_EXST,I10_Q,CY,:,:,:,:,:,:,...,95.95,96.62,97.28,98.06,98.90,99.25,98.51,100.36,99.85,98.72
4,Q,DW_EXST,I10_Q,CZ,:,:,:,:,:,:,...,227.40,230.30,225.40,222.20,220.70,221.60,222.40,225.00,230.30,235.10


In [3]:
path = '../ESTAT_CODELISTS/*.tsv'
files = glob.glob(path)
dataframes = {file.split('/')[-1].replace('.tsv', ''): pd.read_csv(
    file, sep='\t', on_bad_lines='skip') for file in files}

In [4]:
for col in ["freq", "unit", "purchase", "geo"]:
    curr_codelist = dataframes[f"ESTAT_{col.upper()}"]
    df[col] = df[col].map(curr_codelist.set_index('CODE')['Label'])

df.rename(columns={'geo': 'country'}, inplace=True)
df.head()

,freq,purchase,unit,country,2005-Q1,2005-Q2,2005-Q3,2005-Q4,2006-Q1,2006-Q2,...,2022-Q2,2022-Q3,2022-Q4,2023-Q1,2023-Q2,2023-Q3,2023-Q4,2024-Q1,2024-Q2,2024-Q3
0,Quarterly,Purchases of existing dwellings,"Quarterly index, 2010=100",Austria,:,:,:,:,:,:,...,223.26,227.52,217.81,215.58,214.13,212.07,207.28,204.23,211.26,211.98 p
1,Quarterly,Purchases of existing dwellings,"Quarterly index, 2010=100",Belgium,72.31,74.46,78.01,79.97,81.55,83.90,...,143.09,146.87,145.40,146.72,146.18,147.84,149.38,149.82,149.77,152.61 p
2,Quarterly,Purchases of existing dwellings,"Quarterly index, 2010=100",Bulgaria,72.66,75.77,77.96,79.84,83.62,84.99,...,160.96,167.01,169.23,174.59,179.37,182.01,185.80,199.15,205.93,212.00 p
3,Quarterly,Purchases of existing dwellings,"Quarterly index, 2010=100",Cyprus,:,:,:,:,:,:,...,95.95,96.62,97.28,98.06,98.90,99.25,98.51,100.36,99.85,98.72
4,Quarterly,Purchases of existing dwellings,"Quarterly index, 2010=100",Czechia,:,:,:,:,:,:,...,227.40,230.30,225.40,222.20,220.70,221.60,222.40,225.00,230.30,235.10


In [5]:
df.replace(": z", np.nan, inplace=True)
df.replace(":", np.nan, inplace=True)

In [6]:
year_cols = [col for col in df.columns if col.isdigit()]

for col in year_cols:
    df[col] = df[col].astype(str).str.extract(r'(\d+)').astype(float)

In [7]:
df.head()

,freq,purchase,unit,country,2005-Q1,2005-Q2,2005-Q3,2005-Q4,2006-Q1,2006-Q2,...,2022-Q2,2022-Q3,2022-Q4,2023-Q1,2023-Q2,2023-Q3,2023-Q4,2024-Q1,2024-Q2,2024-Q3
0,Quarterly,Purchases of existing dwellings,"Quarterly index, 2010=100",Austria,NaN,NaN,NaN,NaN,NaN,NaN,...,223.26,227.52,217.81,215.58,214.13,212.07,207.28,204.23,211.26,211.98 p
1,Quarterly,Purchases of existing dwellings,"Quarterly index, 2010=100",Belgium,72.31,74.46,78.01,79.97,81.55,83.90,...,143.09,146.87,145.40,146.72,146.18,147.84,149.38,149.82,149.77,152.61 p
2,Quarterly,Purchases of existing dwellings,"Quarterly index, 2010=100",Bulgaria,72.66,75.77,77.96,79.84,83.62,84.99,...,160.96,167.01,169.23,174.59,179.37,182.01,185.80,199.15,205.93,212.00 p
3,Quarterly,Purchases of existing dwellings,"Quarterly index, 2010=100",Cyprus,NaN,NaN,NaN,NaN,NaN,NaN,...,95.95,96.62,97.28,98.06,98.90,99.25,98.51,100.36,99.85,98.72
4,Quarterly,Purchases of existing dwellings,"Quarterly index, 2010=100",Czechia,NaN,NaN,NaN,NaN,NaN,NaN,...,227.40,230.30,225.40,222.20,220.70,221.60,222.40,225.00,230.30,235.10


In [8]:
for col in df.columns:
    if not col.isdigit() and df[col].nunique() == 1:
        df.drop(columns=[col], inplace=True)

In [9]:
df.head()

,purchase,unit,country,2005-Q1,2005-Q2,2005-Q3,2005-Q4,2006-Q1,2006-Q2,2006-Q3,...,2022-Q2,2022-Q3,2022-Q4,2023-Q1,2023-Q2,2023-Q3,2023-Q4,2024-Q1,2024-Q2,2024-Q3
0,Purchases of existing dwellings,"Quarterly index, 2010=100",Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,223.26,227.52,217.81,215.58,214.13,212.07,207.28,204.23,211.26,211.98 p
1,Purchases of existing dwellings,"Quarterly index, 2010=100",Belgium,72.31,74.46,78.01,79.97,81.55,83.90,86.99,...,143.09,146.87,145.40,146.72,146.18,147.84,149.38,149.82,149.77,152.61 p
2,Purchases of existing dwellings,"Quarterly index, 2010=100",Bulgaria,72.66,75.77,77.96,79.84,83.62,84.99,88.87,...,160.96,167.01,169.23,174.59,179.37,182.01,185.80,199.15,205.93,212.00 p
3,Purchases of existing dwellings,"Quarterly index, 2010=100",Cyprus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,95.95,96.62,97.28,98.06,98.90,99.25,98.51,100.36,99.85,98.72
4,Purchases of existing dwellings,"Quarterly index, 2010=100",Czechia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,227.40,230.30,225.40,222.20,220.70,221.60,222.40,225.00,230.30,235.10


In [10]:
df.to_csv("./clean/estat_prc_hpi_q.csv", index=False)